In [ ]:
import pandas as pd
import geopandas as gpd
from siuba import *
import shared_utils

from calitp_data_analysis import get_fs, geography_utils
from segment_speed_utils import helpers, time_series_utils, gtfs_schedule_wrangling
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT, analysis_date

# Scratchpad for displaying excluded shapes on speedmaps

* implemented via `ca_transit_speedmaps/speedmap_utils.py` for now
* TODO: more systematic/clarify cause as speed data gap or shape not running in time period

In [ ]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [ ]:
catalog.speedmap_segments

In [ ]:
analysis_date

In [ ]:
helpers.import_scheduled_shapes?

In [ ]:
shapes = helpers.import_scheduled_shapes(analysis_date, columns=['shape_array_key', 'geometry'])

In [ ]:
trips = helpers.import_scheduled_trips(analysis_date, columns=['shape_array_key', 'shape_id', 'route_id',
                                                               'route_short_name', 'gtfs_dataset_key']).drop_duplicates()

In [ ]:
shapes = shapes.merge(trips, on='shape_array_key')

In [ ]:
helpers.import_scheduled_trips?

In [ ]:
path = f"{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet"

In [ ]:
path

In [ ]:
gdf = gpd.read_parquet(path)

In [ ]:
gdf.columns

In [ ]:
pd.read_parquet('../ca_transit_speed_maps/_rt_progress_2024-12-11.parquet').query('name.str.contains("Muni")')

In [ ]:
sf_speeds = gdf.query('organization_source_record_id == "rechaapWbeffO33OX"')

In [ ]:
sf_speeds

In [ ]:
sf_shapes = shapes.query('schedule_gtfs_dataset_key == "7cc0cb1871dfd558f11a2885c145d144"')

In [ ]:
sf_shapes

In [ ]:
sf_am = sf_speeds.query('time_of_day == "AM Peak"')

In [ ]:
sf_am = sf_am.to_crs(geography_utils.CA_NAD83Albers)

In [ ]:
## shift to right side of road to display direction
sf_am.geometry = sf_am.geometry.apply(shared_utils.rt_utils.try_parallel)
sf_am = sf_am.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1, frequency_col='trips_hr_sch')

In [ ]:
sf_am = sf_am.dissolve()

sf_am.geometry = sf_am.buffer(35)

In [ ]:
sf_am.explore()

In [ ]:
gdf = sf_shapes.overlay(sf_am, how='difference')

In [ ]:
gdf.explore()